In [36]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import activations
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV 
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.layers import Dropout

print(pd.__version__)


1.3.5


In [76]:
df = pd.read_csv("rawpoints/Train82.csv",low_memory=False)
tdf = pd.read_csv("rawpoints/Test82.csv")

In [77]:
df = df.dropna() #removing nan values
tdf = tdf.dropna() #removing nan values
# tdf.head()

In [78]:
# df.head()
train_x = df.loc[:, df.columns != 'ImgPath' ]
train_y = np.array(train_x.loc[:,train_x.columns == 'label'])
train_x = np.array(train_x.loc[:, train_x.columns != 'label' ])
# print(train_x)

# print(train_x)

# df.head()
test_x = tdf.loc[:, tdf.columns != 'ImgPath' ]
test_y = np.array(test_x.loc[:,test_x.columns == 'label'])
test_x = np.array(test_x.loc[:, test_x.columns != 'label' ])
# print(train_x)

# print(test_x)

In [79]:

scaler = preprocessing.StandardScaler().fit(train_x)
train_x = scaler.transform(train_x)
# for i in range(len(train_y)):
    # train_y[i][0]=int(train_y[i][0])
# len(train_x[0])

scaler = preprocessing.StandardScaler().fit(test_x)
test_x = scaler.transform(test_x)



In [80]:
train_y_lst = []
for i in range(len(train_x)):
    lt = np.zeros((82,), dtype=int)
    lt[train_y[i]] = 1
    train_y_lst.append(lt)
train_y_lst = np.array(train_y_lst)
train_y_lst

test_y_lst = []
for i in range(len(test_x)):
    lt = np.zeros((82,), dtype=int)
    lt[test_y[i]] = 1
    test_y_lst.append(lt)
test_y_lst = np.array(test_y_lst)
print(test_x)
# test_y_lst

[[-0.61402961 -0.50192187 -0.71025018 ...  0.21223679  0.91107768
   0.7551523 ]
 [ 0.05822381 -0.03669392  0.15267182 ... -0.26612688 -0.26990379
   0.08987409]
 [ 0.44289044  0.45277137  0.42190437 ...  0.08575432  0.70882581
   0.49558071]
 ...
 [ 0.14393422  0.18932109  0.12659374 ...  0.03518295  0.3935659
   0.08443708]
 [ 0.05399048  0.1655146   0.01195941 ... -0.50988237  0.10165657
  -0.4880183 ]
 [-0.40073809 -0.45736434 -0.36404861 ...  0.65596734  0.18484645
   0.44014383]]


In [81]:
# tf.keras.optimizers.legacy.Adam()
model = tf.keras.models.Sequential([
    Dense(200, activation = 'relu'),
    Dropout(0.2),
    Dense(150, activation = 'relu'),
    Dropout(0.1),
    Dense(100, activation = 'relu'),
    Dropout(0.1),
    Dense(82, activation = 'relu'),
    Dropout(0.1),
    Dense(20, activation = 'relu'),
    Dense(6, activation = 'relu'),
    Dense(82, activation = 'softmax')
    ])
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

history = model.fit(train_x,train_y_lst, validation_data=(test_x, test_y_lst),epochs = 200,batch_size = 16,verbose = 1)

Epoch 1/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0520 - loss: 4.0759 - val_accuracy: 0.1680 - val_loss: 3.1364
Epoch 2/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1780 - loss: 3.0848 - val_accuracy: 0.3412 - val_loss: 2.4843
Epoch 3/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2848 - loss: 2.5708 - val_accuracy: 0.4112 - val_loss: 2.1510
Epoch 4/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3855 - loss: 2.2150 - val_accuracy: 0.4961 - val_loss: 1.8705
Epoch 5/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4557 - loss: 1.9861 - val_accuracy: 0.5771 - val_loss: 1.6759
Epoch 6/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5162 - loss: 1.7659 - val_accuracy: 0.5746 - val_loss: 1.6184
Epoch 7/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5260 - loss: 1.7201 - val_accuracy: 0.5967 - val_loss: 1.5579
Epoch 8/200
762/762 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5570 - loss: 1.5986 - val_accu

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()

# Plot training and validation accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Curves')
plt.legend()

plt.show()

In [ ]:
model.evaluate(train_x, train_y_lst)

In [ ]:
model.evaluate(test_x, test_y_lst)

In [ ]:

from sklearn.metrics import accuracy_score

In [ ]:
y_pred = model.predict(test_x)


In [ ]:
act_y = []
for i in range(len(y_pred)):
    ind = -1;
    max = -1;
    for j in range(len(y_pred[i])):
        if(y_pred[i][j]>max):
            max = y_pred[i][j]
            ind = j;
    act_y.append(ind)
act_y = np.array(act_y).reshape(-1,1)

In [ ]:

act_y

In [ ]:
accuracy_score(test_y,act_y)
